# Wrappers

pddlgym expects certain specific class info 

`StringWrapper` allows us to give this as a string. E.g., `"move(tree1_7:tree1, crafting_table1_5:crafting_table1)"`

`PyperWrapper` for pyperplan compatibility

In [ ]:
# String Wrappers 
## For stringifying stuff to pddlgym

import gym
from pddlgym.structs import Predicate, Literal, TypedEntity, Type

class StringWrapper(gym.Wrapper):
    """
    Allows for providing actions as strings rather than the pddlgym representation
    """
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.current_state = None
        
    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        self.current_state = obs
        return obs

    def step(self,action_str):
        action = self._str_to_action(action_str)
        print(f"Action: {action}", type(action))
        next_state, reward, done, info = self.env.step(action)
        self.current_state = next_state
        return next_state, reward, done, info
    
    def observe(self):
        return self.current_state

    def _str_to_action(self, action_str):
        name = action_str.split("(")[0]
        args = action_str.split("(")[1].split(")")[0].split(",")
        objs = []
        for arg in args:
            arg = arg.replace(" ","")
            obj_type_str = arg.split(":")[1]
            obj_const_str = arg.split(":")[0]

            obj_type = Type(obj_type_str)
            obj_const = obj_type(obj_const_str)
            objs.append(obj_const)
        actionP = Predicate(name, len(args))
        action = Literal(actionP, objs)
        return action
     

In [ ]:
import pddlgym

env = pddlgym.make('PDDLEnvTreasure-v0')
env = StringWrapper(env)
env.reset()
obs = env.observe()
print(obs)
action = "move(r1:r,q3:q)"
obs,_,_,_ = env.step(action)
print(obs)

In [44]:
# pyperplan wrapper 

import gym
import pddlgym.structs as pgym 
import pyperplan.pddl.pddl as pyper


# pyperplan wrapper
class PyperWrapper(gym.Wrapper):
    """
    Allows for translating between pddlgym and pyperplan representations.
    Can run pyperplan solutions in pddlgym
    Can read pddlgym output in pyperplan state representation
    """
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.current_state = None  # going to be in default pddlgym representation
        self.root_object_type = pyper.Type("object",None)
        self.history = [] #In pyperplan format

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        self.current_state = obs
        return self.observe()


    def observe(self, source='pypertask'):
        if source == 'pyperpred':
            return self._obs_pddlgym_to_pyper(self.current_state)
        if source == 'pddlgym':
            return self.current_state
        if source == 'pypertask':
            return self._obs_pddlgym_to_pypertask(self.current_state)
        raise ValueError("source must be either 'pyper' or 'pddlgym'")



    def objects(self):
        """
        Returns all the objects in the current state
        """
        objects = {}
        objs = self.current_state[0].objects
        for o in set(objs):
            key = o.name
            type_name = str(o.var_type)
            value = pyper.Type(type_name, self.root_object_type)
            objects.update({key : value})
        return objects



    def step(self,action):
        action_pddlgym = self._action_pyper_to_pddlgym(action)
        next_state = self.env.step(action_pddlgym)
        self.history.append(action)
        self.current_state = next_state

        return self.observe()

    def get_history(self):
        return self.history

    def _action_pyper_to_pddlgym(self, action):
        """
        (move p3 k1) --> pddlgym representation
        """
        name = action.replace("(","").replace(")","").split(" ")[0]
        objs = set(self.current_state[0].objects)
        args = action.replace("(","").replace(")","").split(" ")[1:]
        arity = len(args)
        typed_args = [ (lambda x: [x for x in objs if x == y][0])(y) for y in args]
        action_pddlgym = pgym.Literal(pgym.Predicate(name, arity), typed_args)
        return action_pddlgym


    def _obs_pddlgym_to_pyper(self, obs):
        predicates = []
        for item in set(obs[0].literals):
            name = str(item).split("(")[0]
            list_args_str = str(item).split("(")[1].split(")")[0].split(",")
            signature = [(x.split(":")[0], [pyper.Type(x.split(":")[1], self.root_object_type)]) for x in list_args_str]
            pred = pyper.Predicate(name, signature)
            predicates.append(pred)
        return predicates

    
    
    def _obs_pddlgym_to_pypertask(self, obs):
        literals = []
        for item in set(obs[0].literals):
            name = str(item).split("(")[0]
            list_args_str = str(item).split("(")[1].split(")")[0].split(",")
            new_args = [x.split(":")[0] for x in list_args_str]
            new_lit = "("+name+" "+" ".join(new_args)+")"
            literals.append(new_lit)
        return frozenset(literals)

    
        # DEPRECATED
    def __get_type(self, constant):
        # look at current state 
        obj_strs = [str(x) for x in set(self.current_state[0].objects)]
        obj_types = [x.split(":")[1] for x in obj_strs if constant+":" in x]
        if len(obj_types) == 1:
            return obj_types[0]
        raise ValueError
    

In [46]:
import pddlgym

env = pddlgym.make('PDDLEnvTreasure-v0')
env = PyperWrapper(env)
obs = env.reset()
print(f"Init: {obs}\n")

action = '(move r1 b3)'
obs= env.step(action)
action = '(move b3 q4)'
obs= env.step(action)
print(f"Final: {obs}\n")

obs = env.observe()
print(f"Observe: {obs}\n")

obs = env.observe(source="pddlgym")
print(f"Observe (pddlgym): {obs}\n")

obs = env.observe(source="pypertask")
print(f"Observe (task): {obs}\n")

obs = env.observe(source="pyperpred")
print(f"Observe (pred): {obs}\n")

objs = env.objects()
print(f"Objects: {objs}")

h = env.get_history()
print(f"History: {h}")


Init: frozenset({'(inworld b3)', '(inworld q5)', '(inworld q2)', '(inworld p5)', '(inworld r1)', '(inworld b1)', '(in s2 b2)', '(inworld p4)', '(inworld b2)', '(at r1)', '(inworld r2)', '(inworld p2)', '(inworld p3)', '(inworld p1)', '(in s1 b1)', '(inworld q1)', '(inworld q3)', '(inworld q4)'})

Final: frozenset({'(inworld b3)', '(inworld q5)', '(inworld q2)', '(inworld p5)', '(inworld r1)', '(inworld b1)', '(in s2 b2)', '(inworld p4)', '(inworld b2)', '(inworld r2)', '(inworld p2)', '(at q4)', '(inworld p3)', '(inworld p1)', '(in s1 b1)', '(inworld q1)', '(inworld q3)', '(inworld q4)'})

Observe: frozenset({'(inworld b3)', '(inworld q5)', '(inworld q2)', '(inworld p5)', '(inworld r1)', '(inworld b1)', '(in s2 b2)', '(inworld p4)', '(inworld b2)', '(inworld r2)', '(inworld p2)', '(at q4)', '(inworld p3)', '(inworld p1)', '(in s1 b1)', '(inworld q1)', '(inworld q3)', '(inworld q4)'})

Observe (pddlgym): (State(literals=frozenset({inworld(p1:p), inworld(q2:q), inworld(p3:p), inworld(q1:

# Checking predicates in states

In [146]:
from pyperplan.pddl.pddl import Predicate, Type

root_object_type = Type("object",None)
def _parse_predicate(pred_str):
        """
        Assume a predicate string
        "(have ?x-t)" or "(have p1-p)" or "(on a-block b-block) or (inworld p2)"
        """
        name = pred_str.replace("(","").replace(")","").split(" ")[0]
        args = pred_str.replace("(","").replace(")","").split(" ")[1:]
        signature = []
        grounded = True
        for arg in args:
            if "-" in arg:
                sym = arg.split("-")[0]
                if "?" in sym:
                    grounded = False
                typing = arg.split("-")[1]
                signature.append((sym, [Type(typing,root_object_type)]))
            else:
                if "?" in arg:
                    raise ValueError("Must provide at least one of constant or a type")
                objs = env.objects() #CHANGE TO self.env                
                typing = objs[arg]
                signature.append((arg, [typing]))
        return Predicate(name, signature)


In [151]:
out = _parse_predicate("(inworld ?x-p)")
print(out.name)
print(out.signature)
print(str(out))

inworld
[('?x', [p])]
inworld[('?x', [p])]


In [163]:
import random
import uuid
import copy

def _assign_symbol(predicate): # ADD SELF
    """
    Replaces ?x with either a hypothetical or with an object from the domain 
    """
    predicate_copy = copy.deepcopy(predicate)
    # Flip objects dictionary to be keyed to type name
    objs = env.objects()  # CHANGE TO SELF.ENV
    type_keyed_objs = {}
    for k,v in objs.items():
        type_keyed_objs[v.name]= type_keyed_objs.get(v.name,[])
        type_keyed_objs[v.name].append(k)
    print(type_keyed_objs)
    
    updated_signature = []
    for term,typing in predicate_copy.signature:
        if "?" in term:
            # check if env objects have an object of type
            if typing[0].name in type_keyed_objs:
                # randomly assign
                used_term = random.choice(type_keyed_objs[typing[0].name])
                updated_signature.append((used_term, [typing[0]]))
            else:
                # hypothetical name 
                hypo_term = typing[0].name+"_"+str(uuid.uuid4())[:8]
                updated_signature.append((hypo_term, [typing[0]]))
        else:
             updated_signature.append((term, [typing[0]])) 
    predicate_copy.signature = updated_signature
    return predicate_copy
                
    
    

In [164]:
_assign_symbol(out)

{'p': ['p4', 'p3', 'p5', 'p2', 'p1'], 'r': ['r1'], 's': ['-r', 's2', 's1', 'r2'], 'b': ['b3', 'b2', 'b1'], 'q': ['q3', 'q4', 'q2', 'q1', 'q5'], 'k': ['k1', 'k2'], 't': ['t1']}


inworld[('p4', [p])]

# Alternative approach 

grounding goal from strings

In [26]:
import random

def _ground_goal( atom_str):
    """
    Given an atom (have ?x-t) or (have t23) or (on ?x-t y24-y)  or (have ?t34-t)
    Return (have t23)
    """

    objs = env.objects()  # CHANGE TO SELF.ENV
    type_keyed_objs = {}
    for k,v in objs.items():
        type_keyed_objs[v.name]= type_keyed_objs.get(v.name,[])
        type_keyed_objs[v.name].append(k)

    name = atom_str.replace("(","").replace(")","").split(" ")[0]
    args = atom_str.replace("(","").replace(")","").split(" ")[1:]
    new_args = []
    for arg in args:
        if "?" in arg:
            if "-" in arg:
                typing = arg.split("-")[1]
                if typing in type_keyed_objs:
                    # Assign random name 
                    new_sym = random.choice(type_keyed_objs[typing])
                    new_args.append(f"{new_sym}")
                else:
                    # Assign hypothetical name 
                    hypo_sym = f"{typing}_{str(uuid.uuid4())[:8]}"
                    new_args.append(f"{hypo_sym}")
            else:
                raise ValueError("Must provide at least one of constant or a type")
        else:
            if "-" in arg:
                new_arg = arg.split("-")[0]
                new_args.append(new_arg)
            else:
                new_args.append(arg)
  
    new_atom = "("+name+" "+" ".join(new_args)+")"
    return new_atom



In [35]:
_ground_goal("have ?x-p)")

'(have p4)'

In [40]:
env.observe(source="pddlgym")

(State(literals=frozenset({inworld(p1:p), inworld(q2:q), inworld(p3:p), inworld(q1:q), inworld(q4:q), inworld(r2:r), inworld(q3:q), inworld(b3:b), inworld(b1:b), in(s2:s,b2:b), inworld(b2:b), inworld(q5:q), in(s1:s,b1:b), inworld(r1:r), inworld(p2:p), inworld(p5:p), inworld(p4:p), at(q4:q)}), objects=frozenset({p1:p, q4:q, s1:s, p2:p, t1:t, k1:k, s2:s, q5:q, b3:b, k2:k, p5:p, q1:q, r2:r, p3:p, b2:b, q3:q, b1:b, r1:r, q2:q, p4:p}), goal=have(t1:t)),
 0.0,
 False,
 {'problem_file': '/home/vsarathy/.cache/pypoetry/virtualenvs/construct-VivzdgmF-py3.8/lib/python3.8/site-packages/pddlgym/pddl/treasure/problem01.pddl',
  'domain_file': '/home/vsarathy/.cache/pypoetry/virtualenvs/construct-VivzdgmF-py3.8/lib/python3.8/site-packages/pddlgym/pddl/treasure.pddl'})

In [48]:
a = {"a", "b", "c"}
" ".join(a)

'a c b'

In [55]:
strings = []
for k,v in env.objects().items():
    strings.append(f"{k} - {v}")
" ".join(strings)

'p1 - p q4 - q k2 - k p5 - p s1 - s q1 - q p2 - p t1 - t k1 - k r2 - r p3 - p b2 - b s2 - s q5 - q q3 - q b1 - b r1 - r q2 - q b3 - b p4 - p'